<a href="https://colab.research.google.com/github/jtuckayo/scratchpad/blob/main/DuckDB_%26_Iceberg_The_Future_of_Lightweight_Data_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: install duckdb and sqlite in this notebook

!pip install duckdb
!pip install pysqlite3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp310-cp310-linux_x86_64.whl size=151320 sha256=79c9edae0fa989c99f4e81f658dddbc65c38f26e789fb1e1962ec2bef5990ac1
  Stored in directory: /root/.cache/pip/wheels/04/55/74/e3b53aeb9ff9e933626ee3938ccad69123828115ae34ce028e
Successfully built pysqlite3


In [5]:
pytable = duckdb.sql("SELECT * FROM 's3://us-prd-motherduck-open-datasets/who_ambient_air_quality/csv/who_ambient_air_quality_database_version_v6_april_2023.csv'").arrow()

In [6]:
pytable

pyarrow.Table
who_region: string
iso3: string
country_name: string
city: string
year: int64
version: string
pm10_concentration: string
pm25_concentration: string
no2_concentration: string
pm10_tempcov: string
pm25_tempcov: string
no2_tempcov: string
type_of_stations: string
reference: string
web_link: string
population: string
population_source: string
latitude: string
longitude: string
who_ms: int64
----
who_region: [["3_Sear","3_Sear","3_Sear","3_Sear","3_Sear",...,"5_Emr","5_Emr","5_Emr","5_Emr","5_Emr"]]
iso3: [["IND","IND","IND","IND","IND",...,"SAU","SAU","SAU","SAU","SAU"]]
country_name: [["India","India","India","India","India",...,"Saudi Arabia","Saudi Arabia","Saudi Arabia","Saudi Arabia","Saudi Arabia"]]
city: [["Chennai","Solapur","Chennai","Hyderabad","Pune",...,"Jizan","Jizan","Jizan","Jizan","Jizan"]]
year: [[2018,2016,2019,2019,2017,...,2014,2013,2012,2011,2010]]
version: [["version 2022","version 2022, version 2018","version 2022","version 2022","version 2022",...,"ver

In [7]:
# prompt: install pyiceberg

!pip install pyiceberg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 10.0 MB/s eta 0:00:00


In [8]:
!mkdir "/content/warehouse"

In [10]:
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "/content/warehouse"
catalog = SqlCatalog(
    "default",
    **{
      "uri":f"sqlite:///{warehouse_path}/pyiceberg_catalog_db",
      "warehouse":f"file://{warehouse_path}",
    },
)

In [11]:
catalog.create_namespace("default")

table = catalog.create_table(
    "default.air_quality",
    schema = pytable.schema,
)

In [12]:
table.append(pytable)
len(table.scan().to_arrow())

41364

In [13]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [14]:
con_ice.sql('FROM air;')

┌────────────┬─────────┬──────────────────────────────────────────────────────┬────────────────────────────────┬───────┬────────────────────────────┬────────────────────┬────────────────────┬───────────────────┬──────────────┬──────────────┬─────────────┬──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬───────────────────┬─────────────┬─────────────┬────────┐
│ who_region │  iso3   │                     country_name                     │              city              │ year  │          version           │ pm10_concentration │ pm25_concentration │ no2_concentration │ pm10_tempcov │ pm25_tempcov │ no2_tempcov │ type_of_stations │                                                                reference  

In [15]:
df = table.scan(row_filter="city == 'Berlin'").to_arrow()

In [16]:
table.overwrite(df)

In [17]:
con_ice = table.scan().to_duckdb(table_name = "air")

In [18]:
con_ice.sql('FROM air;')

┌────────────┬─────────┬──────────────┬─────────┬───────┬──────────────┬────────────────────┬────────────────────┬───────────────────┬──────────────┬──────────────┬─────────────┬───────────────────────────────┬───────────────────────────────┬─────────────────────────────────────────────────────────────────┬────────────┬───────────────────┬─────────────┬─────────────┬────────┐
│ who_region │  iso3   │ country_name │  city   │ year  │   version    │ pm10_concentration │ pm25_concentration │ no2_concentration │ pm10_tempcov │ pm25_tempcov │ no2_tempcov │       type_of_stations        │           reference           │                            web_link                             │ population │ population_source │  latitude   │  longitude  │ who_ms │
│  varchar   │ varchar │   varchar    │ varchar │ int64 │   varchar    │      varchar       │      varchar       │      varchar      │   varchar    │   varchar    │   varchar   │            varchar            │            varchar            │